In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'

In [2]:
file_name = 'normalized_minmax_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, 'group_' + group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
for i in groups_n:
    print(len(i[0]))

2
3
4
5


In [5]:
svd['count'] = 1
svd[:10]

,uid,tid,rating,count
0,0,3,0.478011,1
1,0,7,0.516294,1
2,0,18,0.467991,1
3,0,20,0.486123,1
4,0,31,0.569040,1
5,0,35,0.637375,1
6,0,37,0.519302,1
7,0,43,0.486123,1
8,0,54,0.366112,1
9,0,71,0.548618,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

,tid,count,rating
166,166,4231,677.50
80,80,2248,485.50
457,457,2200,207.75
1364,1364,1977,394.00
6338,6338,1942,181.25
350,350,1927,456.00
7209,7209,1844,510.75
20321,20321,1757,161.75
17377,17377,1667,113.00
3761,3761,1659,452.00


In [7]:
# Linear
# size = int(len(pop_rating) // 5)
# pop_normalized = pd.DataFrame()
# for i in range(5):
#     if i == 0:
#         pop = pop_count[:size].copy()
#     elif i == 4:
#         pop = pop_count[size*4:].copy()
#     else:
#         pop = pop_count[size*i:size*(i+1)].copy()
#     pop['rating'] = (5 - i)
#     pop_normalized = pop_normalized.append(pop)
# pop_normalized = pop_normalized[['tid', 'rating']]

In [8]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
166,166,4231,5.000000
80,80,2248,2.656582
457,457,2200,2.599858
1364,1364,1977,2.336327
6338,6338,1942,2.294966
350,350,1927,2.277239
7209,7209,1844,2.179154
20321,20321,1757,2.076341
17377,17377,1667,1.969983
3761,3761,1659,1.960529


In [9]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
for lambda_ in tqdm(lambdas):
    top_n_size = 20
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    print(avg_precisions)
#     print(sum(avg_precisions) / len(avg_precisions))
#     print('----------------------------------------') 

[0.1675, 0.24814814814814815, 0.32272727272727275, 0.43000000000000005]
[0.16750000000000004, 0.2518518518518519, 0.3272727272727273, 0.43000000000000005]
[0.16833333333333333, 0.2518518518518519, 0.3227272727272728, 0.43]
[0.1733333333333333, 0.25, 0.33181818181818185, 0.43]
[0.18500000000000008, 0.2611111111111111, 0.35000000000000003, 0.44000000000000006]
[0.19333333333333336, 0.2796296296296297, 0.36363636363636365, 0.45999999999999996]


In [195]:
alpha = 0.5
threshold = 0.0
for i in tqdm(range(len(groups_n))):
    
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    
    satisfactions = []
    for j in (range(len(groups))):
        
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        high_ratings_t = np.zeros(top_n_size)
        high_ratings_u = np.zeros(len(group))
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_ratings_t[k] += 1
                    high_ratings_u[l] += 1
        
#         satisfaction_t = 0
#         for k in range(top_n_size):
#             satisfaction_t += math.log(high_ratings_t[k]+1, 10) / math.log(len(group)+1, 10)
#         satisfaction_t /= top_n_size
        
#         satisfaction_u = 0
#         for k in range(len(group)):
#             satisfaction_u += math.log(high_ratings_u[k]+1, 10) / math.log(top_n_size+1, 10)
#         satisfaction_u /= len(group)
        satisfaction_t = math.log(max(high_ratings_t)+1, 10) / math.log(len(group)+1, 10)
        satisfaction_u = math.log(min(high_ratings_u)+1, 10) / math.log(top_n_size+1, 10)
        
        satisfaction = alpha*satisfaction_t + (1-alpha)*satisfaction_u
        satisfactions.append(satisfaction)
    
    avg_satisfaction = 0
    for satisfaction in satisfactions:
        avg_satisfaction += satisfaction
    avg_satisfaction /= len(satisfactions)
    print(avg_satisfaction)
        

0.4384689835799772
0.34093950544457624
0.3358646172870294
0.33769451038646087

